<a href="https://colab.research.google.com/github/IO-2024/IOTs_with_python/blob/main/Ch6.%20%EA%B2%BD%EC%A0%9C%ED%8C%8C%EA%B8%89%ED%9A%A8%EA%B3%BC%20%EB%B6%84%EC%84%9D.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np   # Numpy import
import pandas as pd  # Pandas import


3부문 총, 국산, 수입거래표 불러오기

In [2]:
url_Tt_3 = "https://raw.githubusercontent.com/IO-2024/IOTs_with_python/main/data/Total_Matrix_3_secs.csv"  # 3부문의 총거래표 불러오기
Tt_3 = pd.read_csv(url_Tt_3)

url_Dt_3 = "https://raw.githubusercontent.com/IO-2024/IOTs_with_python/main/data/Domestic_Matrix_3_secs.csv"  # 3부문의 국산거래표 불러오기
Dt_3 = pd.read_csv(url_Dt_3)

url_It_3 = "https://raw.githubusercontent.com/IO-2024/IOTs_with_python/main/data/Imported_Matrix_3_secs.csv"  # 3부문의 수입거래표 불러오기
It_3 = pd.read_csv(url_It_3)

In [3]:
Tt = Tt_3.to_numpy()  # numpy로 불러오기
Dt = Dt_3.to_numpy()
It = It_3.to_numpy()

In [4]:
Tt_1 = Tt[0:10,1:13].astype(float)
Dm_1 = Dt[0:4,1:10].astype(float)
Im_1 = It[0:4,1:10].astype(float)

In [5]:
Ct = Tt_1[-1,0:3]  # 총산출액 벡터
Ct

array([  60., 1792., 2447.])

In [6]:
Ct_m = np.tile(Ct, reps=[3,1]) # 총산출액 벡터를 을 3번 행반복
Ct_m

array([[  60., 1792., 2447.],
       [  60., 1792., 2447.],
       [  60., 1792., 2447.]])

In [7]:
At = Tt_1[0:3,0:3]/Ct_m   # 총투입계수
Ad = Dm_1[0:3,0:3]/Ct_m  # 국산투입계수
Ai = Im_1[0:3,0:3]/Ct_m  # 수입투입계수

print('총투입계수표 : \n', At)
print('국산투입계수표 : \n', Ad)
print('수입투입계수표 : \n', Ai)

df_At = pd.DataFrame(At.round(3),columns = ['농산품', '공산품', '서비스'], index=['농산품', '공산품', '서비스'])  #pandas로 내보내기
df_Ad = pd.DataFrame(Ad.round(3),columns = ['농산품', '공산품', '서비스'], index=['농산품', '공산품', '서비스'])  #pandas로 내보내기
df_Ai = pd.DataFrame(Ai.round(3),columns = ['농산품', '공산품', '서비스'], index=['농산품', '공산품', '서비스'])  #pandas로 내보내기

총투입계수표 : 
 [[0.06666667 0.02120536 0.0044953 ]
 [0.26666667 0.52678571 0.16387413]
 [0.11666667 0.14341518 0.28320392]]
국산투입계수표 : 
 [[0.06666667 0.01729911 0.00408664]
 [0.25       0.328125   0.1315897 ]
 [0.11666667 0.13560268 0.2631794 ]]
수입투입계수표 : 
 [[0.         0.00390625 0.00040866]
 [0.01666667 0.19866071 0.03228443]
 [0.         0.0078125  0.02002452]]


### 물가파급효과 분석


##### - 투입계수의 전치행렬을 이용하여 생산유발계수(Lf)을 이용하여 각종 물가파급효과 분석 가능

In [8]:
At_trans = At.T  # 총투입계수의 전치행렬
Ad_trans = Ad.T  # 국산투입계수의 전치행렬
Ai_trans = Ai.T  # 수입투입계수의 전치행렬

print('총투입계수의 전치행렬 : \n', At_trans, '\n')
print('국산투입계수의 전치행렬 : \n', Ad_trans, '\n')
print('수입투입계수의 전치행렬 : \n', Ai_trans, '\n')

Lf_trans = np.linalg.inv(np.identity(3)-Ad_trans)  # (I-Ad_trans)의 역행렬

print('국산투입계수 전치행렬의 역행렬 : \n', Lf_trans)

총투입계수의 전치행렬 : 
 [[0.06666667 0.26666667 0.11666667]
 [0.02120536 0.52678571 0.14341518]
 [0.0044953  0.16387413 0.28320392]] 

국산투입계수의 전치행렬 : 
 [[0.06666667 0.25       0.11666667]
 [0.01729911 0.328125   0.13560268]
 [0.00408664 0.1315897  0.2631794 ]] 

수입투입계수의 전치행렬 : 
 [[0.         0.01666667 0.        ]
 [0.00390625 0.19866071 0.0078125 ]
 [0.00040866 0.03228443 0.02002452]] 

국산투입계수 전치행렬의 역행렬 : 
 [[1.08092021 0.45201623 0.25433867]
 [0.03012689 1.55662423 0.29124759]
 [0.01137552 0.28050647 1.41060738]]


In [9]:
Lf = np.linalg.inv(np.identity(3)-Ad)  # 생산유발계수
Lf

array([[1.08092021, 0.03012689, 0.01137552],
       [0.45201623, 1.55662423, 0.28050647],
       [0.25433867, 0.29124759, 1.41060738]])

In [10]:
Va_r = Tt_1[-2,0:3]/Ct  # 부가가치율 벡터
Va_m = np.diag(Va_r)    # 부가가치율의 대각행렬

Lv = Va_m@Lf   #부가가치유발계수
Li = Ai@Lf  #수입유발계수

Aw_r = Tt_1[-6,0:3]/Ct # 피용자보수율
Aw_m = np.diag(Aw_r)   # 피용자보수율의 대각행렬
print('피용자보수율의 대각행렬 \n', Aw_m,'\n')

피용자보수율의 대각행렬 
 [[0.11666667 0.         0.        ]
 [0.         0.10993304 0.        ]
 [0.         0.         0.29178586]] 



#### 임금인상의 물가파급효과

In [11]:
delta_all_w = np.array([10,10,10]).reshape([3,1]) # 전부문 10% 임금 인상의 물가파급효과
delta_all_Pw = ((Lf_trans@Aw_m)@delta_all_w).round(3)

delta_par_w = np.array([0,10,0]).reshape([3,1]) # 공산품 부문 10% 임금 인상의 물가파급효과
delta_par_Pw = ((Lf_trans@Aw_m)@delta_par_w).round(3)

print('전부문 10% 임금 인상의 물가파급효과 \n', delta_all_Pw, '\n')
print('공산품 부문 10% 임금 인상의 물가파급효과 \n', delta_par_Pw, '\n')


전부문 10% 임금 인상의 물가파급효과 
 [[2.5  ]
 [2.596]
 [4.438]] 

공산품 부문 10% 임금 인상의 물가파급효과 
 [[0.497]
 [1.711]
 [0.308]] 



#### 내생부분(중간투입품) 가격변동의 물가파급효과

In [12]:
#### 중간투입되는 서비스의 물가변동의 파급효과 분석

Ad_exd_row = np.delete(Ad_trans,2,0).reshape([2,3])  # 서비스(sectors3) 행 삭제
Ad_exd_serv = np.delete(Ad_exd_row,2,1).reshape([2,2])  # 서비스(sectors3) 열 삭제
# print('서비스 부문을 제외한 여타 부문 국산투입계수의 전치행렬 \n', Ad_exd_serv.round(3))

Lf_Ad = np.linalg.inv(np.identity(2)-Ad_exd_serv)
# print('서비스 부문을 제외한 여타 부문 국산투입계수 전치행렬의 역행렬 \n', Lf_Ad.round(3))

A_Pr_sec2 = np.concatenate((Ad_trans[0,2].reshape([1,1]),Ad_trans[1,2].reshape([1,1])),axis=0).reshape([2,1])  # 물가가 변동하는 부문의 국산 투입계수(a31, a32)
# print('서비스 부문의 여타부문으로의 국산투입계수 벡터의 전치행렬 \n', A_Pr_sec2.round(3))

# 서비스 부문 10%  가격 인상의 타부문 물가파급효과

delta_serv_Pw = Lf_Ad@A_Pr_sec2*10
print('서비스 부문 10%  가격 인상의 타부문 물가파급효과 \n', delta_serv_Pw.round(3))

서비스 부문 10%  가격 인상의 타부문 물가파급효과 
 [[1.803]
 [2.065]]


#### 수입농산품 가격변동의 물가파급효과


In [13]:
Imp_Pw = np.array([10,0,0]).reshape([3,1]) # 수입농산품 10% 가격 인상의 물가파급효과
delta_imp_Pw =  ((Lf_trans @Ai_trans)@Imp_Pw).reshape([3,1])

print('수입농산품 10% 가격 인상의 물가파급효과 \n', delta_imp_Pw)

수입농산품 10% 가격 인상의 물가파급효과 
 [[0.01869627]
 [0.06199586]
 [0.01672192]]


### 환율변동 효과

In [14]:
Ex_rate = np.array([10,10,10]).reshape([3,1]) # 대미 환율 10% 상승의 물가파급효과
delta_Ex_rate =  ((Lf_trans @Ai_trans)@Ex_rate).reshape([3,1])

print('대미 환율 10% 상승의 물가파급효과 \n', delta_Ex_rate)

대미 환율 10% 상승의 물가파급효과 
 [[1.26518396]
 [3.43337764]
 [1.33566247]]


### 환율변동과 수입농산품 효과

In [15]:
Ex_arg = np.array([21,10,10]).reshape([3,1]) # 수입 농산품 및 대미 환율 각각 10% 상승의 물가파급효과
delta_Ex_arg =  ((Lf_trans @Ai_trans)@Ex_arg).reshape([3,1])

print('수입 농산품 및 대미 환율 각각 10% 상승의 물가파급효과 \n', delta_Ex_arg )

수입 농산품 및 대미 환율 각각 10% 상승의 물가파급효과 
 [[1.28574987]
 [3.50157309]
 [1.35405659]]


In [16]:
#pandas로 내보내기

df_delta_all_Pw = pd.DataFrame(delta_all_Pw.round(3),columns = ['물가상승률'], index=['농산품', '공산품', '서비스'])  # 모든 부문의 임금 10% 상승
df_delta_par_Pw = pd.DataFrame(delta_par_Pw.round(3),columns = ['물가상승률'], index=['농산품', '공산품', '서비스'])  # 공산품 부문의 임금 10% 상승
df_delta_serv_Pw = pd.DataFrame(delta_serv_Pw.round(3),columns = ['물가상승률'], index=['농산품', '서비스'])        # 서비스 물가 10% 상승
df_delta_imp_Pw = pd.DataFrame(delta_imp_Pw.round(3),columns = ['물가상승률'], index=['농산품', '공산품', '서비스'])   # 수입 농산품 물가 10% 상승
df_delta_Ex_rate = pd.DataFrame(delta_Ex_rate.round(3),columns = ['물가상승률'], index=['농산품', '공산품', '서비스'])   # 환율 10% 상승
df_delta_Ex_arg = pd.DataFrame(delta_Ex_arg.round(3),columns = ['물가상승률'], index=['농산품', '공산품', '서비스'])   # 환율 및 수입농산품 10% 상승

In [17]:
df_delta_all_Pw.to_csv('모든 부문 임금 10% 상승.csv')
df_delta_par_Pw.to_csv('공산품 부문 임금 10% 상승.csv')
df_delta_serv_Pw.to_csv('서비스 물가 10% 상승.csv')
df_delta_imp_Pw.to_csv('수입 농산품 물가 10% 상승.csv')
df_delta_Ex_rate.to_csv('환율 10% 상승.csv')
df_delta_Ex_arg.to_csv('환율 및 수입농산품 10% 상승.csv')

### 산출 변동요인분해

In [18]:
# 2015년 거래표

url_total = "https://raw.githubusercontent.com/Muraka64/IOTs_with_python/main/data/2015_tot_cst_3secs.csv"  # 총거래표(3부문)
Tot_cst_15 = pd.read_csv(url_total)

url_imp = "https://raw.githubusercontent.com/Muraka64/IOTs_with_python/main/data/2015_imp_cst_3secs.csv"  # 수입거래표(3부문)
Imp_cst_15 = pd.read_csv(url_imp)

url_dom = "https://raw.githubusercontent.com/Muraka64/IOTs_with_python/main/data/2015_dom_cst_3secs.csv"  # 국산거래표(3부문)
Dom_cst_15 = pd.read_csv(url_dom)

In [19]:
# 2010년 거래표

url_total = "https://raw.githubusercontent.com/Muraka64/IOTs_with_python/main/data/2010_tot_cst_3secs.csv"  # 2010년 불변 가격기준 총거래표(3부문)
Tot_cst_10 = pd.read_csv(url_total)

url_imp = "https://raw.githubusercontent.com/Muraka64/IOTs_with_python/main/data/2010_imp_cst_3secs.csv"  # 2010년 불변 가격기준 수입거래표(3부문)
Imp_cst_10 = pd.read_csv(url_imp)

url_dom = "https://raw.githubusercontent.com/Muraka64/IOTs_with_python/main/data/2010_dom_cst_3secs.csv"  # 2010년 불변 가격기준 국산거래표(3부문)
Dom_cst_10 = pd.read_csv(url_dom)

In [20]:
S1= Tot_cst_15.iloc[:,1:].to_numpy().astype(float)
I1 = Imp_cst_15.iloc[:,1:].to_numpy().astype(float)
S0 = Tot_cst_10.iloc[:,1:].to_numpy().astype(float)
I0 = Imp_cst_10.iloc[:,1:].to_numpy().astype(float)

In [21]:
SS0 = S0[0:3,0:3]  # 총거래표의 내생부문
SS1 = S1[0:3,0:3]
SS1.shape

(3, 3)

In [22]:
II0 = I0[0:3,0:3]  # 수입거래표의 내생부문
II1 = I1[0:3,0:3]

In [23]:
DD0 =SS0 - II0  # 국산거래표의 내생부문
DD1 =SS1 - II1
DD0

array([[  4.,  29.,   9.],
       [ 17., 510., 235.],
       [  5., 209., 427.]])

In [24]:
TM0 = I0[0:3,8]  # 수입거래표의 총수요계(총수입) 벡터
TM1 = I1[0:3,8]
TM0

array([ 10., 417.,  72.])

In [25]:
IM0 = I0[0:3,3]  # 수입거래표의 중간수요계(중간재수입) 벡터
IM1 = I1[0:3,3]
IM0

array([  8., 325.,  48.])

In [26]:
IF0 = I0[0:3,7]  # 수입거래표의 최종수요계(최종재수입) 벡터
IF1 = I1[0:3,7]
IF0

array([ 2., 92., 24.])

In [27]:
SFC0 = S0[0:3,4]  # 총거래표의 소비 벡터
SFC1 = S1[0:3,4]
SFC0

array([ 16., 166., 740.])

In [28]:
SFI0 = S0[0:3,5] # 총거래표의 투자 벡터
SFI1 = S1[0:3,5]
SFI0

array([  3., 138., 310.])

In [29]:
SF0 = S0[0:3,7] # 총거래표의 최종수요계
SF1 = S1[0:3,7]
SF0

array([  20.,  761., 1129.])

In [30]:
EX0 = S0[0:3,6] # 총거래표의 수출 벡터
EX1 = S1[0:3,6]
EX0

array([  1., 457.,  79.])

In [31]:
DF0 = SF0 - EX0 # 총거래표의 내수(최종수요계 - 수출) 벡터
DF1 = SF1 - EX1
DF0

array([  19.,  304., 1050.])

In [32]:
C0 = S0[-1,0:3].T.reshape([3,1])  # 부문별 총산출액 벡터
C1 = S1[-1,0:3].T.reshape([3,1])
C0

array([[  60.],
       [1431.],
       [1746.]])

In [33]:
TC0 = C0.sum(axis=0)  # 전부문 총산출액계
TC1 = C1.sum(axis=0)
TC0

array([3237.])

In [34]:
CT0 = np.tile(C0, reps=[1,3])   # Ct벡터의 반복 행렬(3x3) 생성
CT1 = np.tile(C1, reps=[1,3])
CT1

array([[  62.,   62.,   62.],
       [1713., 1713., 1713.],
       [2058., 2058., 2058.]])

In [35]:
AS0 = SS0/CT0.T # 총투입계수표
AS1 = SS1/CT1.T
# np.isnan(AS_t)

In [36]:
AD0 = DD0/CT0.T # 국산투입계수표
AD1 = DD1/CT1.T

In [37]:
II0 = AS0-AD0 # 수입투입계수표
II1 = AS1-AD1

In [38]:
M0 = IF0/DF0 # 최종재수입/내수(소비+투자)
M0 = np.diag(M0)
M1 = IF1/DF1
M1 = np.diag(M1)

In [39]:
RD0 = np.linalg.inv(np.eye(3) - AD0) # 생산유발계수
RD1 = np.linalg.inv(np.eye(3) - AD1)

In [40]:
AM0 = II0/AS0 #조정항목
AM0 = AM0*AS1
AM1 = II1/AS1
AM1 = AM1*AS0

In [41]:
AA = AS1 - AS0  # 투입계수 변화
AS1_f = pd.DataFrame(AS1)
AS1_f.to_csv('AS1_f.csv')

#### 산출액성장요인분해
-  파쉐 및 라스파이레스 방식으로 산출액 변동을 분해하여 평균으로 성장요인을 분해

In [42]:
# 총산출액 변동요인별 기여율(파쉐 방식)

TQS=(RD1@(np.eye(3)-M1))@(DF1-DF0).reshape([3,1]) # 국내최종수요 변동효과
TQS

array([[  2.38510022],
       [ 61.62837687],
       [215.97946683]])

In [43]:
TTE= RD1@(EX1-EX0).reshape([3,1]) # 수출 변동효과
TTE

array([[  5.36844407],
       [256.66813339],
       [ 86.76931313]])

In [44]:
T3 = ((RD1@(M1 - M0))@DF0).reshape([3,1]) # 최종재 수입대체효과
T3 = T3*-1
T4 = ((RD1@(AM1 - II0)))@C0[0:3,] # 중간재 수입대체효과
T4 = T4*-1
T34 = T3 + T4
TI = np.concatenate((T34, T3, T4), axis=1)
T34

array([[ -2.2227443 ],
       [-43.08542286],
       [-21.98890467]])

In [45]:
T5 = (RD1@(AA-(II1-AM1)))@C0 # 기술구조 변화효과
T5.shape
T5

array([[-3.53079999],
       [ 6.78891261],
       [31.2401247 ]])

In [46]:
DCT0 = C1-C0  # 총산출액 실질변동액
DCT1 = TQS + TTE + T3 + T4 + T5 # 총산출액 변동요인별 합
DCT = DCT0-DCT1
DCT0

array([[  2.],
       [282.],
       [312.]])

In [47]:
SUM = np.concatenate((DCT0, DCT1, DCT, TQS, TTE, T3, T4, T5), axis=1)
SUM1 = np.sum(SUM, axis=0).reshape([1,8])
PGD1015_f = (np.concatenate((SUM, SUM1), axis=0))
print('총산출액 변동요인별 금액 : \n ', PGD1015_f.round(3))

총산출액 변동요인별 금액 : 
  [[  2.      2.     -0.      2.385   5.368  -1.505  -0.717  -3.531]
 [282.    282.     -0.     61.628 256.668 -19.78  -23.305   6.789]
 [312.    312.     -0.    215.979  86.769 -13.936  -8.053  31.24 ]
 [596.    596.     -0.    279.993 348.806 -35.221 -32.076  34.498]]


In [48]:
PGD1015_ff = pd.DataFrame(PGD1015_f.round(3),columns = ['15-10산출액(A)', '요인별합계(B)', 'B-A', '국내최종수요 변동효과', '수출 변동효과', '최종재 수입대체효과','중간재 수입대체효과', '기술구조 변화효과'],

                         index=['농산품', '공산품', '서비스', '계'])  # pandas로 내보내기

In [49]:
PGD1015_ff

,15-10산출액(A),요인별합계(B),B-A,국내최종수요 변동효과,수출 변동효과,최종재 수입대체효과,중간재 수입대체효과,기술구조 변화효과
농산품,2.0,2.0,-0.0,2.385,5.368,-1.505,-0.717,-3.531
공산품,282.0,282.0,-0.0,61.628,256.668,-19.780,-23.305,6.789
서비스,312.0,312.0,-0.0,215.979,86.769,-13.936,-8.053,31.240
계,596.0,596.0,-0.0,279.993,348.806,-35.221,-32.076,34.498


In [50]:
Ctbr_Pa =  PGD1015_f[:,3:]
sm_gap_Pa = PGD1015_f[:,1].reshape([4,1])  # 요인별 합계
sm_gap1_Pa = np.tile(sm_gap_Pa, reps=[1,5]) # 요인별 합계를 4x5으로 반복
Ctrb_f_Pa = ((Ctbr_Pa/sm_gap1_Pa).round(3))*100  # 변동요인별 기여율(파쉐식)
print('변동요인별 기여율(파쉐식) \n', Ctrb_f_Pa, '\n')

변동요인별 기여율(파쉐식) 
 [[ 119.3  268.4  -75.3  -35.9 -176.5]
 [  21.9   91.    -7.    -8.3    2.4]
 [  69.2   27.8   -4.5   -2.6   10. ]
 [  47.    58.5   -5.9   -5.4    5.8]] 



In [51]:
Ctrb_ff_Pa = pd.DataFrame(Ctrb_f_Pa.round(1),columns = ['국내최종수요증감액', '수출', '최종재수입','중간재수입', '기술변화'],

                         index=['농산품', '공산품', '서비스', '계'])  # pandas로 내보내기
Ctrb_ff_Pa

,국내최종수요증감액,수출,최종재수입,중간재수입,기술변화
농산품,119.3,268.4,-75.3,-35.9,-176.5
공산품,21.9,91.0,-7.0,-8.3,2.4
서비스,69.2,27.8,-4.5,-2.6,10.0
계,47.0,58.5,-5.9,-5.4,5.8


In [52]:
# 총산출액 변동요인별 기여율(라스파이레스 방식)

TQS=(RD0@(np.eye(3)-M0))@(DF1-DF0).reshape([3,1])
TQS

array([[  2.5574703 ],
       [ 63.27713436],
       [214.29752996]])

In [53]:
TTE= RD0@(EX1-EX0).reshape([3,1]) # 수출효과
TTE

array([[  6.12654761],
       [260.10573822],
       [ 86.70357226]])

In [54]:
T3 = ((RD0@(M1 - M0))@DF1).reshape([3,1]) # 최종재수입대체효과
T3 = T3*-1
T4 = ((RD0@(II1 - AM0)))@C1[0:3,] # 중간재수입대체효과
T4 = T4*-1
T34 = T3 + T4
TI = np.concatenate((T34, T3, T4), axis=1)
T34

array([[ -2.51646901],
       [-49.48964707],
       [-25.04596103]])

In [55]:
T5 = (RD0@(AA-(AM0-II0)))@C1 # 기술변화효과
T5

array([[-4.16754891],
       [ 8.10677449],
       [36.04485881]])

In [56]:
SUM = np.concatenate((DCT0, DCT1, DCT, TQS, TTE, T3, T4, T5), axis=1)
SUM1 = np.sum(SUM, axis=0).reshape([1,8])
LGD1015_f = (np.concatenate((SUM, SUM1), axis=0))
print('총산출액 변동요인별 금액 : \n ', LGD1015_f.round(3))

총산출액 변동요인별 금액 : 
  [[  2.      2.     -0.      2.557   6.127  -1.619  -0.898  -4.168]
 [282.    282.     -0.     63.277 260.106 -21.282 -28.208   8.107]
 [312.    312.     -0.    214.298  86.704 -15.458  -9.588  36.045]
 [596.    596.     -0.    280.132 352.936 -38.358 -38.694  39.984]]


In [57]:
LGD1015_ff = pd.DataFrame((LGD1015_f).round(3),columns = ['15-10산출액(A)', '요인별합계(B)', 'B-A', '국내최종수요증감액', '수출', '최종재수입','중간재수입', '기술변화'],

                         index=['농산품', '공산품', '서비스', '계'])  # pandas로 내보내기

In [58]:
LGD1015_ff

,15-10산출액(A),요인별합계(B),B-A,국내최종수요증감액,수출,최종재수입,중간재수입,기술변화
농산품,2.0,2.0,-0.0,2.557,6.127,-1.619,-0.898,-4.168
공산품,282.0,282.0,-0.0,63.277,260.106,-21.282,-28.208,8.107
서비스,312.0,312.0,-0.0,214.298,86.704,-15.458,-9.588,36.045
계,596.0,596.0,-0.0,280.132,352.936,-38.358,-38.694,39.984


In [59]:
Ctbr_La =  LGD1015_f[:,3:]
sm_gap_La = LGD1015_f[:,1].reshape([4,1])  # 요인별 합계
sm_gap1_La = np.tile(sm_gap_La, reps=[1,5]) # 요인별 합계를 4x5으로 반복
Ctrb_f_La = ((Ctbr_La/sm_gap1_La).round(3))*100  # 변동요인별 기여율(라스파이레스식)
print('변동요인별 기여율(라스파이레스) \n', Ctrb_f_La, '\n')

변동요인별 기여율(라스파이레스) 
 [[ 127.9  306.3  -80.9  -44.9 -208.4]
 [  22.4   92.2   -7.5  -10.     2.9]
 [  68.7   27.8   -5.    -3.1   11.6]
 [  47.    59.2   -6.4   -6.5    6.7]] 



In [60]:
Ctrb_ff_La = pd.DataFrame(Ctrb_f_La.round(1),columns = ['국내최종수요증감액', '수출', '최종재수입','중간재수입', '기술변화'],

                         index=['농산품', '공산품', '서비스', '계'])  # pandas로 내보내기
Ctrb_ff_La

,국내최종수요증감액,수출,최종재수입,중간재수입,기술변화
농산품,127.9,306.3,-80.9,-44.9,-208.4
공산품,22.4,92.2,-7.5,-10.0,2.9
서비스,68.7,27.8,-5.0,-3.1,11.6
계,47.0,59.2,-6.4,-6.5,6.7


In [61]:

PGD1015_ff.to_csv('산출액변동요인_파쉐.csv', encoding="utf-8-sig")
LGD1015_ff.to_csv('산출액변동요인_라스파이레스.csv', encoding="utf-8-sig")
Ctrb_ff_Pa.to_csv('항목별 기여율_파쉐.csv', encoding="utf-8-sig")
Ctrb_ff_La.to_csv('항목별 기여율_라스파이레스.csv', encoding="utf-8-sig")